# Plot data from the VIEWS API, CM level

In [ ]:
# TODO: Needs refactoring and incorporating automatic production of pgm maps for the same data selection. PGM is currently a separate file. 

In [ ]:
import requests
import pandas as pd
import json
import os
from pathlib import Path

#import matplotlib.pyplot as plt

# External
import geopandas as gpd
import sqlalchemy as sa

# VIEWS
from viewser.operations import fetch
from views_mapper2.mapper2 import *
from views_mapper2.BBoxWriter import *
from views_mapper2.dictionary_writer import *
from ingester3.config import source_db_path

## Select forecasts to fetch

In [ ]:
# List the dataset specified in the URL above

model = 'fatalities002'
release = '2024_09_t01'
dataset = f'{model}_{release}'

# Define the URL you want to fetch data from. 

loa = 'cm'

FORECASTS = f'https://api.viewsforecasting.org/{dataset}/{loa}'
FORECASTS 


## Set prediction step (month) to analyze

In [ ]:
step = 3 # set step to plot

## Fetch forecasts from VIEWS API

In [ ]:
# Request function
r = requests.get(FORECASTS)

#The set page size 
PAGE_SIZE=1000

#Define the full request 
req_url=FORECASTS #+'/'+str()+'?'+'pagesize='+str(PAGE_SIZE)
    
master_list=[]
r=requests.get(req_url)
page_data=r.json()

list(page_data)

In [ ]:
# The loop to keep fetching pages until all data is accessed 

master_list+=page_data['data']
    
while page_data['next_page'] != '':
    r=requests.get(page_data['next_page'])
    page_data=r.json()

    master_list+=page_data['data']
        
forecasts=pd.DataFrame(master_list)
forecasts

In [ ]:
model

In [ ]:
# Filter out the relevant data columns, and rename them where needed

if model == 'fatalities001':
        columns_to_keep=['country_id','month_id','sc_cm_sb_main']
        forecasts = forecasts[columns_to_keep].copy()
        forecasts.rename(columns={'sc_cm_sb_main':'main_mean_ln'}, inplace=True)
        print('Here is the filtered and renamed dataframe for the fatalities001 model:')
        
if model == 'fatalities002':
        columns_to_keep=['country_id','month_id','main_mean', 'main_mean_ln']
        forecasts = forecasts[columns_to_keep].copy()
        print('Here is the filtered dataframe for the fatalities002 model:')

forecasts

In [ ]:
# Add relevant transforms of the main output column

forecasts['main_mean_ln2'] = np.log1p(forecasts['main_mean_ln']).astype(float)
print('Added a column for ln2 version of main_mean_ln. See below.')


if model == 'fatalities001':
    forecasts['main_mean'] = np.expm1(forecasts['main_mean_ln'])
    print('Added a column for the linear version of main_mean_ln. See below.')
forecasts

In [ ]:
# Generate month to plot based on first month of selected dataset and the step chosen above

t = (forecasts['month_id'].min())-1
month_to_plot = t + step 
month_to_plot

## Set output folder

In [ ]:
import os
home = os.path.expanduser("~")
#dropbox = home+'/ViEWS Dropbox/VIEWS research collaboration/Dissemination/Working Papers/Prediction Competition Preprint Maps/'
desktop = home+'/Desktop/' # (Toggle on to save to desktop) Change to your directory
#forecasts.to_csv(desktop+'forecasts.csv', index=False)

## Fetch geometries for plotting and merge with dataframe

In [ ]:
engine = sa.create_engine(source_db_path) 
gdf_ci_master = gpd.GeoDataFrame.from_postgis(
    "SELECT id as country_id, name, in_africa, in_me, geom FROM prod.country",
    engine,
    geom_col='geom'
)
gdf_ci_master = gdf_ci_master.to_crs(4326)
gdf_c = gdf_ci_master.copy()


In [ ]:
gdf_c

In [ ]:
data = pd.merge(forecasts, gdf_c, on="country_id", how="left")
data

In [ ]:
gdf = gpd.GeoDataFrame(data, geometry="geom")
gdf

## Plot settings

### Overwrite relevant Mapper2 functions for flexibility in settings

In [ ]:
from views_mapper2.mapper2 import Mapper2

def custom_add_colorbar(
    self,
    cmap,
    vmin,
    vmax,
    location="right",
    size="5%",
    pad=0.1,
    transparency=None,
    labelsize=10,
    tickparams=None,
):
    norm = plt.Normalize(vmin, vmax)
    if isinstance(cmap, str):
        cmap = plt.get_cmap(cmap)
    cmap = color.force_alpha_colormap(cmap=cmap, alpha=transparency)
    scalar_to_rgba = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    divider = make_axes_locatable(self.ax)
    self.cax = divider.append_axes(location, size, pad)
    self.cax.tick_params(labelsize=labelsize)

    tickvalues = (
        list(tickparams.values()) if tickparams is not None else None
    )
    self.cbar = plt.colorbar(
        scalar_to_rgba, cax=self.cax, ticks=tickvalues
    )
    if tickparams is not None:
        self.cbar.set_ticklabels(list(tickparams.keys()))
    return self

# Monkey patch the method
Mapper2.add_colorbar = custom_add_colorbar




### Set labels for plotted months in the file paths

In [ ]:

#if month_to_plot == 546:
 #   month_in_filepath = '546(June2025)'

### Plot and save the data

In [ ]:
filtered_gdf = gdf[gdf['month_id'] == month_to_plot]  

region = 'ame' #options are 'ame', 'me' or 'globe'

cmap = ['rainbow', 'binary']

for cmaps in cmap:
    test_map = Mapper2(
        width=10,   # dimension width
        height=10,  # dimension height
        frame_on=True,
        bbox=bbox_from_cid(f'{region}'), 
    ).add_layer(
        gdf=filtered_gdf,  
        cmap=cmaps,
        transparency=1,
        background=None, 
        map_dictionary=dictionary_stand_1p_10k, # changed based on column
        edgecolor="black",  # border color choice
        linewidth=0.1,  # line size choice
        column="main_mean_ln", 
    )
    
    # Jim procedure for country border creation using the geometries inherent to views3
  #  ax = test_map.ax
   # gdf_c.plot(ax=ax, edgecolor='black', linewidth=0.05, facecolor='None')

    # Print map status
    print(f'{cmaps} map all done!')

    # Save the map with appropriate filename
    test_map.save(f'{desktop}/map_{dataset}_{loa}_s{step}_for_m{month_to_plot}_ln1_{region}_{cmaps}', dpi=350)